# Homework 1: CS-GY 9223
## Exploring 20 NewsGroups

In this homework, you will write a D3 Visualization in Javascript and integrate it in Jupyter Notebook.

The goal of this exercise is to explore the **20 News Groups dataset**, a popular machine learning dataset that contains news articles grouped in 20 topics. Your visualization should receive the dataset and display a bar chart with the top most frequent words in the dataset. The user should be able to filter the data based on topic (for example, by clicking in checkboxes, selecting from a drop down menu, etc.). The user should also be able to export the selected documents from the selected topic back to Python (using a button).

In summary, your visualization should have the following capabilities:
- Display a bar chart with the top K words in the document collection
- Enable the user to filter the documents based on topic, and display a bar chart with the frequency of the top K words from that topic.
- Export the documents from the selected news topic back to python (as a list of strings).
- The visualization has to be integrated with python. The API should have two functions:
  - `plot_top_words(documents, K) # plot top K words using D3 and Javascript`
  - `get_exported_documents() # get the exported documents back to python`
  
Example of the resulting visualization:
<img src="https://github.com/yeb2Binfang/CS_9223_Visualization_for_ML/blob/main/HW/HW1/HW_Vis.png?raw=1" width = "500px" height="100px"/>

### Accessing the data

The data should be accessed from sklearn. In this section we show an example of code for accessing the documents and the document classes.

In [1]:
# Fetching the data
from sklearn.datasets import fetch_20newsgroups
import numpy as np
newsgroups = fetch_20newsgroups(subset='test')

# getting the topic ids
topic_idx = np.array(newsgroups.target, dtype=int)

# getting the unique topic names
topic_names = np.array(newsgroups.target_names)

# getting the list of documents
documents = list(newsgroups.data)

# getting the list of topics (in the same order as documents)
topics = list(topic_names[topic_idx])

These are the 20 topics in the dataset:

In [2]:
topic_names

array(['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc',
       'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware',
       'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles',
       'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt',
       'sci.electronics', 'sci.med', 'sci.space',
       'soc.religion.christian', 'talk.politics.guns',
       'talk.politics.mideast', 'talk.politics.misc',
       'talk.religion.misc'], dtype='<U24')

The documents and document topics are assigned to the variables *documents* and *topics*. We print some document examples below.

## count word frequence
this function is used to count the word frequences, I will split the word such as  I'll to "I" and "ll"

For this dunction, it will return the array of top k words and the array of each word's frequences in accending order

In [41]:
import operator

def count_word_frequence(document,k):
  doc = document
  ##create a dist
  data = {}
  words = doc.replace("(",' ').replace(")",' ').replace("'",' ').replace("*",' ').replace(".",' ').replace("/",' ').replace("-",' ').replace(":",' ').replace("<",' ').replace(">",' ').replace("[",' ').replace("]",' ').replace("!",' ').replace("?",' ').replace(",",' ').strip("\n").lower()
  ##split the text
  words1 = words.split()
  ##count text
  for word in words1:
    if word in data:
      data[word] +=1
    else:
      data[word] = 1
  ###
  #sorted the dist by value in acending order
  sorted_fre = np.array(sorted(data.items(), key=operator.itemgetter(1),reverse=True))

  num_of_words = sorted_fre.shape[0]
  fre = np.zeros(num_of_words)
  words = []

  for i in range(0,num_of_words):
    fre[i] = int(sorted_fre[i][1])
    w = sorted_fre[i][0]
    words.append(w)
    
  words = np.array(words)
  

  return words[:k].tolist(),fre[:k].tolist()



In [4]:
words,words_fre = count_word_frequence(document=documents[1],k=5)
print(words)
print(words_fre)

['i', 'face', 'rick', 'miller', 'x']
[10.0, 4.0, 3.0, 3.0, 3.0]


## drop down menu

In [5]:
from IPython.display import display, HTML
import json

In [6]:
%%javascript
require.config({
    paths: {
        d3: "https://d3js.org/d3.v6.min"
     }
});

require(["d3"], function(d3) {
    window.d3 = d3;
});

<IPython.core.display.Javascript object>

https://www.d3-graph-gallery.com/graph/line_select.html

http://bl.ocks.org/williaster/10ef968ccfdc71c30ef8

this two websites will teach you how to make it

In [7]:
from string import Template

In [8]:
topic_names_JS = topic_names.tolist()


In [ ]:
# template = Template("""
# <html>
# <head>
# </head>
# <body>
# <div id='vis-container'></div>
# <script>
#     var topic_names = $topic;
#     var words = $words;
#     var fre = $words_fre;
#     console.log(words);
#     console.log(fre);

#     var vis = function (wordsMap) {
#         //visualize data
#         // Define dimensions of vis
#         var margin = { top: 30, right: 50, bottom: 30, left: 50 },
#             width = 550 - margin.left - margin.right,
#             height = 250 - margin.top - margin.bottom;
    
#         // Make x scale
#         var xScale = d3.scale.ordinal()
#             .domain(words)
#             .rangeRoundBands([0, width], 0.1);
    
#         // Make y scale, the domain will be defined on bar update
#         var yScale = d3.scale.linear()
#             .range([height, 0]);
                
#         // Create canvas
#         var canvas = d3.select("#vis-container")
#             .append("svg")
#             .attr("width", width + margin.left + margin.right)
#             .attr("height", height + margin.top + margin.bottom)
#             .append("g")
#             .attr("transform", "translate(" + margin.left + "," + margin.top + ")");
        
#         // Make x-axis and add to canvas
#         var xAxis = d3.svg.axis()
#             .scale(xScale)
#             .orient("bottom");
        
#         canvas.append("g")
#             .attr("class", "x axis")
#             .attr("transform", "translate(0," + height + ")")
#             .call(xAxis);
        
        
#         // Make y-axis and add to canvas
#         var yAxis = d3.svg.axis()
#             .scale(yScale)
#             .orient("left"); 
        
#         var yAxisHandleForUpdate = canvas.append("g")
#             .attr("class", "y axis")
#             .call(yAxis);
        
#         //text freuence position update
#         yAxisHandleForUpdate.append("text")
#             .attr("transform", "rotate(-90)")
#             .attr("y", 6)
#             .attr("dy", ".71em")
#             .style("text-anchor", "end")
#             .text("Frequences");
            
#         //update bar value
#         var updateBars = function (data) {
#             // First update the y-axis domain to match data
            
#             yScale.domain([0, d3.max(data) + 1]);
#             yAxisHandleForUpdate.call(yAxis);

#             var bars = canvas.selectAll(".bar").data(data);

#             // Add bars for new data
#             bars.enter()
#                 .append("rect")
#                 .attr("class", "bar")
#                 .attr("x", function (d, i) { return xScale(words[i]); })
#                 .attr("width", xScale.rangeBand())
#                 .attr("y", function (d, i) { return yScale(d); })
#                 .attr("height", function (d, i) { return height - yScale(d); });

#             // Update old ones, already have x / width from before
#             bars
#                 .transition().duration(250)
#                 .attr("y", function (d, i) { return yScale(d); })
#                 .attr("height", function (d, i) { return height - yScale(d); });
#             //Remove old ones
#             bars.exit().remove();
#         };
        
        
        
    
#         //var initialData = wordsMap
#         var dropdown = d3.select("#vis-container")
#             .insert("select", "svg");
#         //.on("change", dropdownChange);

#         dropdown.selectAll("option")
#             .data(topic_names)
#             .enter()
#             .append("option")
#             .attr("value", function (d, i) {
#                 return $topic[i] + $topic[i].slice(1, $topic[i].length);
                
#             })
#             .text(function (d, i) {
#                 return $topic[i];
#             });

#         var initialData = fre;
#         updateBars(initialData);
#     };

#     var readData = function (data) {
#         var wordsMap = {};
#         data.forEach(function (d, i) {
#             var w = words[i];

#             wordsMap[w] = fre[i];
#             console.log(w);
#             console.log(wordsMap[w]);


#         });


#         vis(wordsMap);
#     };

#     readData(words);   
  
# </script>
# </body>

# </html>
# """)
# my_html = template.safe_substitute(topic = topic_names_JS, words=words_JS,words_fre=words_fre)
# display(HTML(my_html))

## visulaize the data

In [ ]:
template = Template("""
<html>
<head>
</head>
<body>
<div id='vis-container'></div>
<script>
    var topic_names = $topic;
    var words = $words;
    var fre = $words_fre;
    

    var vis = function (wordsMap) {
        //visualize data
        // Define dimensions of vis
        var margin = { top: 30, right: 50, bottom: 30, left: 50 },
            width = 550 - margin.left - margin.right,
            height = 250 - margin.top - margin.bottom;
    
        // Make x scale
        var xScale = d3.scale.ordinal()
            .domain(words)
            .rangeRoundBands([0, width], 0.1);
    
        // Make y scale, the domain will be defined on bar update
        var yScale = d3.scale.linear()
            .range([height, 0]);
                
        // Create canvas
        var canvas = d3.select("#vis-container")
            .append("svg")
            .attr("width", width + margin.left + margin.right)
            .attr("height", height + margin.top + margin.bottom)
            .append("g")
            .attr("transform", "translate(" + margin.left + "," + margin.top + ")");
        
        // Make x-axis and add to canvas
        var xAxis = d3.svg.axis()
            .scale(xScale)
            .orient("bottom");
        
        canvas.append("g")
            .attr("class", "x axis")
            .attr("transform", "translate(0," + height + ")")
            .call(xAxis);
        
        
        // Make y-axis and add to canvas
        var yAxis = d3.svg.axis()
            .scale(yScale)
            .orient("left"); 
        
        var yAxisHandleForUpdate = canvas.append("g")
            .attr("class", "y axis")
            .call(yAxis);
        
        //text freuence position update
        yAxisHandleForUpdate.append("text")
            .attr("transform", "rotate(-90)")
            .attr("y", 6)
            .attr("dy", ".71em")
            .style("text-anchor", "end")
            .text("Frequences");
            
        //update bar value
        var updateBars = function (data) {
            // First update the y-axis domain to match data
            
            yScale.domain([0, d3.max(data) + 1]);
            yAxisHandleForUpdate.call(yAxis);

            var bars = canvas.selectAll(".bar").data(data);

            // Add bars for new data
            bars.enter()
                .append("rect")
                .attr("class", "bar")
                .attr("x", function (d, i) { return xScale(words[i]); })
                .attr("width", xScale.rangeBand())
                .attr("y", function (d, i) { return yScale(d); })
                .attr("height", function (d, i) { return height - yScale(d); });

            // Update old ones, already have x / width from before
            bars
                .transition().duration(250)
                .attr("y", function (d, i) { return yScale(d); })
                .attr("height", function (d, i) { return height - yScale(d); });
            //Remove old ones
            bars.exit().remove();
        };
        
        
        
    
        //var initialData = wordsMap
        var dropdown = d3.select("#vis-container")
            .insert("select", "svg");
        //.on("change", dropdownChange);

        dropdown.selectAll("option")
            .data(topic_names)
            .enter()
            .append("option")
            .attr("value", function (d, i) {
                return $topic[i] + $topic[i].slice(1, $topic[i].length);
                
            })
            .text(function (d, i) {
                return $topic[i];
            });

        var initialData = fre;
        updateBars(initialData);
    };

    var readData = function (data) {
        var wordsMap = {};
        data.forEach(function (d, i) {
            var w = words[i];

            wordsMap[w] = fre[i];
            console.log(w);
            console.log(wordsMap[w]);


        });


        vis(wordsMap);
    };

    readData(words);   
  
</script>
</body>

</html>
""")
my_html = template.safe_substitute(topic = topic_names_JS, words=words_JS,words_fre=words_fre)
display(HTML(my_html))

In [4]:
template = Template("""
<!DOCTYPE html>
<meta charset="utf-8">

<!-- Load d3.js -->
<script src="https://d3js.org/d3.v4.js"></script>

<!-- Initialize a select button -->
<select id="selectButton"></select>

<!-- Create a div where the graph will take place -->
<div id="my_dataviz"></div>

<!-- Color Scale -->
<script src="https://d3js.org/d3-scale-chromatic.v1.min.js"></script>

<script>

// set the dimensions and margins of the graph
var margin = {top: 10, right: 100, bottom: 30, left: 30},
    width = 460 - margin.left - margin.right,
    height = 400 - margin.top - margin.bottom;

// append the svg object to the body of the page
var svg = d3.select("#my_dataviz")
  .append("svg")
    .attr("width", width + margin.left + margin.right)
    .attr("height", height + margin.top + margin.bottom)
  .append("g")
    .attr("transform",
          "translate(" + margin.left + "," + margin.top + ")");

//Read the data
d3.csv("https://raw.githubusercontent.com/holtzy/D3-graph-gallery/master/DATA/data_connectedscatter.csv", function(data) {

    // List of groups (here I have one group per column)
    var allGroup = ["valueA", "valueB", "valueC"]

    // add the options to the button
    d3.select("#selectButton")
      .selectAll('myOptions')
     	.data(allGroup)
      .enter()
    	.append('option')
      .text(function (d) { return d; }) // text showed in the menu
      .attr("value", function (d) { return d; }) // corresponding value returned by the button

    // A color scale: one color for each group
    var myColor = d3.scaleOrdinal()
      .domain(allGroup)
      .range(d3.schemeSet2);

    // Add X axis --> it is a date format
    var x = d3.scaleLinear()
      .domain([0,10])
      .range([ 0, width ]);
    svg.append("g")
      .attr("transform", "translate(0," + height + ")")
      .call(d3.axisBottom(x));

    // Add Y axis
    var y = d3.scaleLinear()
      .domain( [0,20])
      .range([ height, 0 ]);
    svg.append("g")
      .call(d3.axisLeft(y));

    // Initialize line with group a
    var line = svg
      .append('g')
      .append("path")
        .datum(data)
        .attr("d", d3.line()
          .x(function(d) { return x(+d.time) })
          .y(function(d) { return y(+d.valueA) })
        )
        .attr("stroke", function(d){ return myColor("valueA") })
        .style("stroke-width", 4)
        .style("fill", "none")

    // A function that update the chart
    function update(selectedGroup) {

      // Create new data with the selection?
      var dataFilter = data.map(function(d){return {time: d.time, value:d[selectedGroup]} })

      // Give these new data to update line
      line
          .datum(dataFilter)
          .transition()
          .duration(1000)
          .attr("d", d3.line()
            .x(function(d) { return x(+d.time) })
            .y(function(d) { return y(+d.value) })
          )
          .attr("stroke", function(d){ return myColor(selectedGroup) })
    }

    // When the button is changed, run the updateChart function
    d3.select("#selectButton").on("change", function(d) {
        // recover the option that has been chosen
        var selectedOption = d3.select(this).property("value")
        // run the updateChart function with this selected option
        update(selectedOption)
    })

})


</script>
""")
my_html = template.safe_substitute()
display(HTML(my_html))

## dropdown menu

In [14]:
topic_names_List = topic_names.tolist()

In [16]:
words_list = ['I','face','milk','education']
words_fre_list = [6,3,2,1]
print(words_list)
print(max(words_fre_list))

['I', 'face', 'milk', 'education']
6


In [66]:
template = Template("""
<!DOCTYPE html>
<meta charset="utf-8">
<body>

<!-- Load d3.js -->
<script src="https://d3js.org/d3.v4.js"></script>

<!-- Initialize a select button -->
<select id="selectButton"></select>

<!-- Create a div where the graph will take place -->
<div id="my_dataviz"></div>

<!-- Color Scale -->
<script src="https://d3js.org/d3-scale-chromatic.v1.min.js"></script>

<script>
// set the dimensions and margins of the graph
var margin = {top: 20, right: 30, bottom: 40, left: 90},
    width = 460 - margin.left - margin.right,
    height = 400 - margin.top - margin.bottom;

// append the svg object to the body of the page
var svg = d3.select("#my_dataviz")
  .append("svg")
    .attr("width", width + margin.left + margin.right)
    .attr("height", height + margin.top + margin.bottom)
  .append("g")
    .attr("transform",
          "translate(" + margin.left + "," + margin.top + ")");

// List of groups (here I have one group per column)
    var topic = $topic
    var words = $words
    var words_fre = $words_fre
    var max = words_fre[0]


  //data vis
  var vis = function(topic,words,words_fre){
    // add the options to the button
    d3.select("#selectButton")
      .selectAll('myOptions')
     	.data(topic)
      .enter()
    	.append('option')
      .text(function (d) { return d; }) // text showed in the menu
      .attr("value", function (d) { return d; }) // corresponding value returned by the button
  
  // Add X axis
  var x = d3.scaleLinear()
    .domain([0, max])
    .range([ 0, width]);

  svg.append("g")
    .attr("transform", "translate(0," + height + ")")
    .call(d3.axisBottom(x))
    .selectAll("text")
      .attr("transform", "translate(-10,0)rotate(-45)")
      .style("text-anchor", "end");

    // Y axis
  var y = d3.scaleBand()
    .range([ 0, height ])
    .domain(words)
    .padding(.1);

  svg.append("g")
    .call(d3.axisLeft(y))

    //Bars
  svg.selectAll("myRect")
    .data(words)
    .enter()
    .append("rect")
    .attr("x", x(0) )
    .attr("y", function(d) { return y(words); })   
    .attr("height", y.bandwidth() )
    

  //Bars
  svg.selectAll("myRect")
    .data(words_fre)
    .enter()
    .append("rect")
    .attr("x", x(0) )
    .attr("y", function(d) { return y(words); })
    .attr("width", function(d) { return x(words_fre); })
    .attr("height", y.bandwidth() )
    .attr("fill", "#69b3a2")
    
  };


  vis(topic,words,words_fre);
</script>
</body>

""")
my_html = template.safe_substitute(topic=topic_names_List,words=words_list,words_fre = words_fre_list)
display(HTML(my_html))

In [67]:
dict1 = []
for i in range(len(words_fre_list)):
  dic = {'words':words_list[i],'fre':words_fre_list[i]}
  dict1.append(dic)
  


In [69]:
print(dict1)

[{'words': 'I', 'fre': 6}, {'words': 'face', 'fre': 3}, {'words': 'milk', 'fre': 2}, {'words': 'education', 'fre': 1}]


##data dict
the data format

{

  'topic1':[{word:'I',fre:'3'},{word:'love',fre:3]}],

  'topic2'...
  
}

In [55]:
data_dict = {}

for i in range(len(topic_names_JS)):
  data_dict[topic_names_JS[i]] = []
  for j in range(5):
    dic = {'word':data_dict_words_and_wordFre[topic_names_JS[i]]['words'][j],'fre':data_dict_words_and_wordFre[topic_names_JS[i]]['words_fre'][j]}
    data_dict[topic_names_JS[i]].append(dic)


In [56]:
print(data_dict['comp.sys.ibm.pc.hardware'])

[{'word': 'the', 'fre': 51.0}, {'word': 'of', 'fre': 24.0}, {'word': 'to', 'fre': 22.0}, {'word': 'and', 'fre': 20.0}, {'word': 'in', 'fre': 18.0}]


In [57]:
data_dict_words_and_wordFre = {}
for i in range(len(topic_names_List)):
  words,words_fre = count_word_frequence(document=documents[i],k=5)
  data_dict_words_and_wordFre[topic_names_List[i]] = {'words':words,'words_fre':words_fre}

In [58]:
print(data_dict_words_and_wordFre['alt.atheism']['words'][0])
print(data_dict_words_and_wordFre['alt.atheism']['words_fre'])

the
[8.0, 4.0, 4.0, 3.0, 3.0]


## drop down and vis

In [29]:
template = Template("""
<html>
<head>
</head>
<body>
    <!-- Load d3.js -->
    <script src="https://d3js.org/d3.v4.js"></script>

    <!-- Initialize a select button -->
    <select id="selectButton"></select>

    <!-- Create a div where the graph will take place -->
    <div id="my_dataviz"></div>

    <!-- Color Scale -->
    <script src="https://d3js.org/d3-scale-chromatic.v1.min.js"></script>
    <script>
      // set the dimensions and margins of the graph
      var margin = {top: 20, right: 30, bottom: 40, left: 90},
          width = 460 - margin.left - margin.right,
          height = 400 - margin.top - margin.bottom;

      // append the svg object to the body of the page
      var svg = d3.select("#my_dataviz")
        .append("svg")
        .attr("width", width + margin.left + margin.right)
        .attr("height", height + margin.top + margin.bottom)
        .append("g")
        .attr("transform",
              "translate(" + margin.left + "," + margin.top + ")");

      // List of groups (here I have one group per column)
      var topic = $topic
      var words = $words
      var words_fre = $words_fre
      var max = words_fre[0]
      var dict = $dict1;

      var data = dict1['alt.atheism'];

      //data vis
      var vis = function(topic,words,words_fre){
        // add the options to the button
        d3.select("#selectButton")
          .selectAll('myOptions')
          .data(topic)
          .enter()
          .append('option')
          .text(function (d) { return d; }) // text showed in the menu
          .attr("value", function (d) { return d; }) // corresponding value returned by the button   
      };
      vis(topic,words,words_fre);
    </script>
</body>
</html>
""")
my_html = template.safe_substitute(topic=topic_names_List,words=words_list,words_fre = words_fre_list,dict1=data_dict)
display(HTML(my_html))

In [ ]:
      // Add X axis
      var x = d3.scaleLinear()
        .domain([0, max])
        .range([ 0, width]);

      svg.append("g")
        .attr("transform", "translate(0," + height + ")")
        .call(d3.axisBottom(x))
        .selectAll("text")
        .attr("transform", "translate(-10,0)rotate(-45)")
        .style("text-anchor", "end");

      // Y axis
      var y = d3.scaleBand()
        .range([ 0, height ])
        .domain(words)
        .padding(.1);

      svg.append("g")
        .call(d3.axisLeft(y))

      //Bars
      svg.selectAll("myRect")
        .data(data1)
        .enter()
        .append("rect")
        .attr("x", x(0) )
        .attr("y", function(d) { return y(d.word); })
        .attr("width", function(d) { return x(d.fre); })
        .attr("height", y.bandwidth() )
        .attr("fill", "#69b3a2")

In [93]:
template = Template("""
<html>

<head>
</head>

<body>
  <!-- Load d3.js -->
  <script src="https://d3js.org/d3.v4.js"></script>

  <!-- Initialize a select button -->
  <select id="selectButton"></select>

  <!-- Create a div where the graph will take place -->
  <div id="my_dataviz"></div>

  <p id="demo"></p>

  <!-- Color Scale -->
  <script src="https://d3js.org/d3-scale-chromatic.v1.min.js"></script>

  <script>
    // set the dimensions and margins of the graph
    var margin = {top: 20, right: 30, bottom: 40, left: 90},
        width = 460 - margin.left - margin.right,
        height = 400 - margin.top - margin.bottom;

    // append the svg object to the body of the page
    var svg = d3.select("#my_dataviz")
                .append("svg")
                .attr("id","graph")
                .attr("width", width + margin.left + margin.right)
                .attr("height", height + margin.top + margin.bottom)
                .append("g")
                .attr("transform",
                "translate(" + margin.left + "," + margin.top + ")");

    var name = 'alt.atheism';
    // List of groups (here I have one group per column)
    var topic = $topic;
    
    //var dict = [{word: 'I', fre: 2}, {word: 'face', fre: 1}];

    var dict1 = $dict1;
    var dict2 = $dict2;
    var data1 = dict1[name];

    var words = dict2[name]['words'];
    console.log(words);
    
    var words_fre = dict2[name]['words_fre'];
    var max = words_fre[0]
    //console.log(data1);
    //console.log(dict);           

    //data vis
    var vis = function(topic,words,words_fre){
      // add the options to the button
      d3.select("#selectButton")
        .selectAll('myOptions')
        .data(topic)
        .enter()
        .append('option')
        .text(function (d) { return d; }) // text showed in the menu
        .attr("value", function (d) { return d; }) // corresponding value returned by the button
    
      // Add X axis
      var x = d3.scaleLinear()
        .domain([0, max])
        .range([ 0, width]);

      svg.append("g")
        .attr("transform", "translate(0," + height + ")")
        .call(d3.axisBottom(x))
        .selectAll("text")
          .attr("transform", "translate(-10,0)rotate(-45)")
          .style("text-anchor", "end");

      // Y axis
      var y = d3.scaleBand()
        .range([ 0, height ])
        .domain(words)
        .padding(.1);

      svg.append("g")
        .call(d3.axisLeft(y));

      //Bars
      svg.selectAll("myRect")
        .data(data1)
        .enter()
        .append("rect")
        .attr("id","Norect")
        .attr("x", x(0) )
        .attr("y", function(d) { return y(d.word); })
        .attr("width", function(d) { return x(d.fre); })
        .attr("height", y.bandwidth() )
        .attr("fill", "#69b3a2");
        
    };
    vis(topic,words,words_fre);
    
  </script>
</body>
</html>
""")
my_html = template.safe_substitute(topic=topic_names_List,dict1=data_dict,dict2=data_dict_words_and_wordFre)
display(HTML(my_html))

In [38]:
print(words_list)

['I', 'face', 'milk', 'education']


##update

In [150]:
template = Template("""
<html>

<head>
</head>

<body>
  <!-- Load d3.js -->
  <script src="https://d3js.org/d3.v4.js"></script>

  <!-- Initialize a select button -->
  <select id="selectButton"></select>

  <!-- Create a div where the graph will take place -->
  <div id="my_dataviz"></div>

  <p id="demo"></p>

  <!-- Color Scale -->
  <script src="https://d3js.org/d3-scale-chromatic.v1.min.js"></script>

  <script>
    // set the dimensions and margins of the graph
    var margin = {top: 20, right: 30, bottom: 40, left: 90},
        width = 460 - margin.left - margin.right,
        height = 400 - margin.top - margin.bottom;

    // append the svg object to the body of the page
    var svg = d3.select("#my_dataviz")
                .append("svg")
                .attr("id","graph")
                .attr("width", width + margin.left + margin.right)
                .attr("height", height + margin.top + margin.bottom)
                .append("g")
                .attr("transform",
                "translate(" + margin.left + "," + margin.top + ")");

    var name = 'alt.atheism';
    // List of groups (here I have one group per column)
    var topic = $topic;
    

    var dict1 = $dict1;
    var dict2 = $dict2;
    var data1 = dict1[name];

    var words = dict2[name]['words'];
    console.log(words);
    
    var words_fre = dict2[name]['words_fre'];
    var max = words_fre[0]
    //console.log(data1);
    //console.log(dict);           

    //data vis
    var vis = function(topic,words,words_fre){
      // add the options to the button
      d3.select("#selectButton")
        .selectAll('myOptions')
        .data(topic)
        .enter()
        .append('option')
        .text(function (d) { return d; }) // text showed in the menu
        .attr("value", function (d) { return d; }) // corresponding value returned by the button
    
      // Add X axis
      var x = d3.scaleLinear()
        .domain([0,10])
        .range([ 0, width]);

      var xAxis = svg.append("g")
        .attr("transform", "translate(0," + height + ")")
        .call(d3.axisBottom(x))
        .selectAll("text")
          .attr("transform", "translate(-10,0)rotate(-45)")
          .style("text-anchor", "end");

      // Y axis
      var y = d3.scaleBand()
        .range([ 0, height ])
        .domain(words)
        .padding(.1);

      var yAxis = svg.append("g")
        .call(d3.axisLeft(y));

      //Bars
      var bar = svg.selectAll("myRect")
        .data(data1)
        .enter()
        .append("rect")
        .attr("id","Norect")
        .attr("x", x(0) )
        .attr("y", function(d) { return y(d.word); })
        .attr("width", function(d) { return x(d.fre); })
        .attr("height", y.bandwidth() )
        .attr("fill", "#69b3a2");

     function update(name){
        

        //update x axis
         words_fre = dict2[name]['words_fre'];
         max = words_fre[0]
         console.log(words_fre);
         console.log(max);
         x.domain([0,max])
         //console.log(x.domain([0,max]))
         xAxis.call(d3.axisBottom(x))

         //update y axis
         words = dict2[name]['words'];
         y.domain(words);
         console.log(words);
         yAxis.call(d3.axisLeft(y));


      }
     // When the button is changed, run the updateChart function
      d3.select("#selectButton").on("change", function(d) {
        // recover the option that has been chosen
        var selectedOption = d3.select(this).property("value")
        document.getElementById("demo").innerHTML = "You selected: " + selectedOption;
        // run the updateChart function with this selected option
        update(selectedOption);
    })
        
    };
    vis(topic,words,words_fre);
    
  </script>
</body>
</html>
""")
my_html = template.safe_substitute(topic=topic_names_List,dict1=data_dict,dict2=data_dict_words_and_wordFre)
display(HTML(my_html))

In [ ]:
 function update(name){
        

        //update x axis
         words_fre = dict2[name]['words_fre'];
         max = words_fre[0]
         console.log(words_fre);
         console.log(max);
         x.domain([0.max]);
         xAxis.call(d3.axisBottom(x))


      }

      // When the button is changed, run the updateChart function
      d3.select("#selectButton").on("change", function(d) {
        // recover the option that has been chosen
        var selectedOption = d3.select(this).property("value")
        document.getElementById("demo").innerHTML = "You selected: " + selectedOption;
        // run the updateChart function with this selected option
        update(selectedOption);
    })